In [ ]:
# Install the Cassandra python driver
!pip install cassandra-driver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 43.5 MB/s eta 0:00:00


In [ ]:
# Import the necessary libraries
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

In [4]:


cloud_config= {
  'secure_connect_bundle': 'secure-connect-learner.zip'
}


with open("learner-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

if session:
  print('Connected!')
else:
  print("An error occurred.")

Connected!


In [5]:


# Create Bronze Table
session.execute("""
CREATE TABLE IF NOT EXISTS example.bronze_sales (
    order_id UUID PRIMARY KEY,
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_priority TEXT,
    order_date DATE,
    ship_date DATE,
    units_sold INT,
    unit_price DOUBLE,
    unit_cost DOUBLE,
    total_revenue DOUBLE,
    total_cost DOUBLE,
    total_profit DOUBLE
);
""")
print("Bronze table created.")


Bronze table created.


In [6]:
# Create Silver Table
session.execute("""
CREATE TABLE IF NOT EXISTS example.silver_sales (
    order_id UUID PRIMARY KEY,
    region TEXT,
    country TEXT,
    item_type TEXT,
    sales_channel TEXT,
    order_date DATE,
    ship_date DATE,
    units_sold INT,
    unit_price DOUBLE,
    total_revenue DOUBLE,
    total_cost DOUBLE,
    profit_margin DOUBLE
);
""")
print("Silver table created.")


Silver table created.


In [7]:
# Create Gold Table
session.execute("""
CREATE TABLE IF NOT EXISTS example.gold_revenue_by_region (
    region TEXT PRIMARY KEY,
    total_revenue DOUBLE
);
""")
print("Gold table for revenue by region created.")




Gold table for revenue by region created.


In [9]:
# Connect to the example Keyspace
session = cluster.connect()

# Load the CSV data into a pandas DataFrame
import pandas as pd
df = pd.read_csv('sales_100.csv')


insert_query = session.prepare("INSERT INTO example.gold_revenue_by_region (region, total_revenue) VALUES (?, ?)")


for index, row in df.iterrows():
    session.execute(insert_query, (row['Region'], row['TotalRevenue']))

print("Data inserted successfully.")



Data inserted successfully.


In [ ]:
# Insert data from DataFrame into the Bronze table
insert_query_bronze = session.prepare("""
INSERT INTO example.bronze_sales (
    order_id, region, country, item_type, sales_channel, order_priority,
    order_date, ship_date, units_sold, unit_price, unit_cost,
    total_revenue, total_cost, total_profit
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for index, row in df.iterrows():
    session.execute(insert_query_bronze, (
        uuid.uuid4(),
        row['Region'],
        row['Country'],
        row['Item Type'],
        row['Sales Channel'],
        row['Order Priority'],
        datetime.strptime(row['Order Date'], '%m/%d/%Y'),
        datetime.strptime(row['Ship Date'], '%m/%d/%Y'),
        int(row['UnitsSold']),
        float(row['UnitPrice']),
        float(row['UnitCost']),
        float(row['TotalRevenue']),
        float(row['TotalCost']),
        float(row['TotalProfit'])
    ))
print("Data inserted into Bronze table.")


Data inserted into Bronze table.


In [ ]:
# Calculate Profit Margin
df['ProfitMargin'] = (df['TotalProfit'] / df['TotalRevenue']) * 100

# Insert data from DataFrame into the Silver table
insert_query_silver = session.prepare("""
INSERT INTO example.silver_sales (
    order_id, region, country, item_type, sales_channel, order_date,
    ship_date, units_sold, unit_price, total_revenue, total_cost, profit_margin
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
""")

for index, row in df.iterrows():
    session.execute(insert_query_silver, (
        uuid.uuid4(),
        row['Region'],
        row['Country'],
        row['Item Type'],
        row['Sales Channel'],
        datetime.strptime(row['Order Date'], '%m/%d/%Y'),
        datetime.strptime(row['Ship Date'], '%m/%d/%Y'),
        int(row['UnitsSold']),
        float(row['UnitPrice']),
        float(row['TotalRevenue']),
        float(row['TotalCost']),
        float(row['ProfitMargin'])
    ))
print("Data inserted into Silver table.")


Data inserted into Silver table.


In [ ]:
# Select the data from the gold table
rows = session.execute("SELECT * FROM example.gold_revenue_by_region")

for row in rows:
    print(row.region, row.total_revenue )

# Close the session
session.shutdown()

Australia and Oceania 1168581.7
Europe 1509220.9
Middle East and North Africa 4205821.41
Central America and the Caribbean 44224.2
Asia 2196359.34
Sub-Saharan Africa 2149107.66
North America 348496.72


In [ ]:
# Fetch data from the Bronze table
rows_bronze = session.execute("SELECT * FROM example.bronze_sales")

for row in rows_bronze:
    print(
        row.order_id, row.region, row.country, row.item_type,
        row.sales_channel, row.order_priority, row.order_date,
        row.ship_date, row.units_sold, row.unit_price,
        row.total_revenue, row.total_cost, row.total_profit
    )

print("Fetched data from Bronze table.")


c4aacbab-ea48-4fee-8661-415c3ff1074a Europe Netherlands Fruits Online L 2016-11-19 2016-11-30 9887 9.33 92245.71 68418.04 23827.67
89c27ba8-0b0e-4b7b-90f1-fc7ee73e6276 Europe France Cosmetics Online M 2015-12-07 2016-01-18 5758 437.2 2517397.6 1516254.14 1001143.46
815341e7-ce7d-46c1-9628-40f2d898c2cd Central America and the Caribbean Dominica Beverages Online L 2012-06-12 2012-07-18 6301 47.45 298982.45 200308.79 98673.66
fcf7626e-728c-4316-b211-c81a5d27d78a Europe Greece Cereal Online H 2015-08-22 2015-10-08 8674 205.7 1784241.8 1015812.14 768429.66
ab3ccc76-56a8-4e04-b347-259ae5088ad3 Europe Estonia Household Offline H 2011-09-01 2011-10-21 9976 668.27 6666661.52 5013339.04 1653322.48
d1502913-0187-42e4-b065-1c4f94f0daa9 Central America and the Caribbean Belize Personal Care Online H 2015-06-14 2015-08-02 3533 81.73 288752.09 200215.11 88536.98
d5a3d3dc-7ac6-496a-94ce-4d284182606c Middle East and North Africa Lebanon Meat Online L 2017-03-08 2017-04-18 8770 421.89 3699975.3 3198331.

In [ ]:
# Fetch data from the Silver table
rows_silver = session.execute("SELECT * FROM example.silver_sales")

for row in rows_silver:
    print(
        row.order_id, row.region, row.country, row.item_type,
        row.sales_channel, row.order_date, row.ship_date,
        row.units_sold, row.unit_price, row.total_revenue,
        row.total_cost, row.profit_margin
    )

print("Fetched data from Silver table.")


c162cb7a-8d9e-4113-8064-75a62e32998b Australia and Oceania East Timor Cereal Offline 2015-12-07 2015-12-30 259 205.7 53276.3 30331.49 43.06757413709285
d84bb2c4-7f06-46ec-9288-9886ef305dfb Sub-Saharan Africa Uganda Personal Care Online 2014-06-19 2014-07-21 451 81.73 36860.23 25558.17 30.661935641747213
c4025fbe-925f-42fb-b682-f9fe5fdf62e4 Europe Romania Cereal Online 2015-04-16 2015-04-18 7337 205.7 1509220.9 859236.07 43.06757413709285
a679c862-74c4-490d-afd0-a44fe04b2c96 Australia and Oceania Papua New Guinea Meat Offline 2015-05-15 2015-06-04 360 421.89 151880.4 131288.4 13.558036455000119
64582bea-6443-461a-a914-2c78c3df488a Sub-Saharan Africa Uganda Cosmetics Online 2015-02-28 2015-03-15 6031 437.2 2636753.2 1588143.23 39.76898444647758
0a224ac2-8b6a-485c-8cf1-7eb24c7a7d2f Europe Finland Household Offline 2016-01-15 2016-01-31 8148 668.27 5445063.96 4094695.92 24.7998563454891
ad958e0a-5927-4776-989e-6a80fb893388 Europe United Kingdom Cosmetics Online 2015-05-01 2015-05-16 1038 4

In [ ]:

session.execute("""
CREATE TABLE IF NOT EXISTS example.gold_top_products_units (
    item_type TEXT PRIMARY KEY,
    total_units_sold INT
);
""")
print("Gold table for top products by total units sold created.")


rows = session.execute("""
SELECT item_type, SUM(units_sold) AS total_units_sold
FROM example.bronze_sales
GROUP BY item_type;
""")

for row in rows:
    session.execute("""
    INSERT INTO example.gold_top_products_units (item_type, total_units_sold)
    VALUES (%s, %s)
    """, (row.item_type, row.total_units_sold))


In [ ]:

session.execute("""
CREATE TABLE IF NOT EXISTS example.gold_profit_by_country (
    country TEXT PRIMARY KEY,
    total_profit DOUBLE
);
""")
print("Gold table for profit by country created.")


rows = session.execute("""
SELECT country, SUM(total_profit) AS total_profit
FROM example.bronze_sales
GROUP BY country;
""")

for row in rows:
    session.execute("""
    INSERT INTO example.gold_profit_by_country (country, total_profit)
    VALUES (%s, %s)
    """, (row.country, row.total_profit))
